In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

from wordcloud import WordCloud
#if encounter error: No module named 'wordcloud'
#run the line below

import folium
from folium.plugins import HeatMap
#if encounter error: No module named 'folium'
#run the line below

import plotly.offline as pyo
import plotly.graph_objs as go
#if encounter error: No module named 'plotly'
#run the line below

import seaborn as sns
sns.set_style('whitegrid')

In [2]:
# Import the dataset
airbnb_data = pd.read_csv("PredictPrice.csv")
airbnb_df = pd.DataFrame(airbnb_data)
airbnb_data.head(5)

,id,new_host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,bedrooms,beds,number_of_reviews,availability_365,price
0,50646.0,65.0,f,Bukit Timah,Central Region,1.33432,103.78521,Private room,1.0,1.0,18.0,365.0,80.0
1,71609.0,65.0,f,Tampines,East Region,1.34537,103.95887,Private room,2.0,3.0,20.0,365.0,179.0
2,71896.0,65.0,f,Tampines,East Region,1.34754,103.95958,Private room,1.0,1.0,24.0,365.0,82.0
3,71903.0,65.0,f,Tampines,East Region,1.34531,103.96100,Private room,1.0,2.0,47.0,365.0,82.0
4,275343.0,77.0,f,Bukit Merah,Central Region,1.29015,103.80814,Private room,1.0,1.0,20.0,341.0,52.0


In [3]:
airbnb_data[['price','new_host_acceptance_rate', 'bedrooms', 'beds', 'number_of_reviews', 'availability_365']].describe()

,price,new_host_acceptance_rate,bedrooms,beds,number_of_reviews,availability_365
count,3671.000000,3671.000000,3671.000000,3671.000000,3671.000000,3671.000000
mean,206.755652,56.200490,1.182784,1.791882,11.777172,225.244348
std,370.691181,36.966118,0.767236,2.168451,31.865084,145.968915
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,70.000000,30.000000,1.000000,1.000000,0.000000,81.000000
50%,135.000000,65.000000,1.000000,1.000000,1.000000,311.000000
75%,220.000000,91.000000,1.000000,2.000000,7.000000,362.000000
max,8900.000000,100.000000,8.000000,46.000000,369.000000,365.000000


__we can see a huge difference in the range of values present in our numerical features:
price,beds,number of reviews and availability_365. We will use scaling to fix this problem__
__to be done during the model building phase__

things to do be done before model building
1) Normalization is a scaling technique in which values are shifted and rescaled so that they end up ranging between 0 and 1. It is also known as Min-Max scaling. Feature scaling to be done later when tuning model for comparsion
2) Convert categorical variable into numbers some  ML libraries do not take categorical variables as input. Label encoder (neigbourhood and region)

In [4]:
airbnb_df

,id,new_host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,bedrooms,beds,number_of_reviews,availability_365,price
0,50646.0,65.0,f,Bukit Timah,Central Region,1.334320,103.785210,Private room,1.0,1.0,18.0,365.0,80.0
1,71609.0,65.0,f,Tampines,East Region,1.345370,103.958870,Private room,2.0,3.0,20.0,365.0,179.0
2,71896.0,65.0,f,Tampines,East Region,1.347540,103.959580,Private room,1.0,1.0,24.0,365.0,82.0
3,71903.0,65.0,f,Tampines,East Region,1.345310,103.961000,Private room,1.0,2.0,47.0,365.0,82.0
4,275343.0,77.0,f,Bukit Merah,Central Region,1.290150,103.808140,Private room,1.0,1.0,20.0,341.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3666,53905492.0,100.0,f,Bukit Merah,Central Region,1.286680,103.834440,Private room,1.0,2.0,0.0,358.0,185.0
3667,53905653.0,100.0,f,Bukit Merah,Central Region,1.285092,103.833326,Private room,1.0,2.0,0.0,361.0,215.0
3668,53910005.0,0.0,f,Marine Parade,Central Region,1.302760,103.901580,Private room,1.0,1.0,0.0,365.0,48.0
3669,53910434.0,30.0,f,Kallang,Central Region,1.300063,103.859894,Entire home/apt,2.0,3.0,0.0,363.0,529.0


In [5]:
#remove rows that are zero
airbnb_df = airbnb_df.loc[(airbnb_df['price'] > 0)]

In [6]:
#label encoding
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
airbnb_df['neighbourhood_cleansed_Cat'] = le.fit_transform(airbnb_df['neighbourhood_cleansed'])
airbnb_df['neighbourhood_group_cleansed_Cat'] = le.fit_transform(airbnb_df['neighbourhood_group_cleansed'])
airbnb_df['room_type_cat'] = le.fit_transform(airbnb_df['room_type'])
airbnb_df['host_is_superhost'] = le.fit_transform(airbnb_df['host_is_superhost'])

new_df=airbnb_df
new_df = new_df.drop(columns=['neighbourhood_cleansed','neighbourhood_group_cleansed','room_type','id','latitude','longitude'])

C:\Users\liang\AppData\Local\Temp/ipykernel_5420/2540961786.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\liang\AppData\Local\Temp/ipykernel_5420/2540961786.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\liang\AppData\Local\Temp/ipykernel_5420/2540961786.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [7]:
new_df.head

<bound method NDFrame.head of       new_host_acceptance_rate  host_is_superhost  bedrooms  beds  \
0                         65.0                  0       1.0   1.0   
1                         65.0                  0       2.0   3.0   
2                         65.0                  0       1.0   1.0   
3                         65.0                  0       1.0   2.0   
4                         77.0                  0       1.0   1.0   
...                        ...                ...       ...   ...   
3666                     100.0                  0       1.0   2.0   
3667                     100.0                  0       1.0   2.0   
3668                       0.0                  0       1.0   1.0   
3669                      30.0                  0       2.0   3.0   
3670                      67.0                  0       1.0   1.0   

      number_of_reviews  availability_365  price  neighbourhood_cleansed_Cat  \
0                  18.0             365.0   80.0             

In [8]:
##log transfromation
new_df.availability_365 += 0.000000001
new_df['availability_365'] = np.log10(new_df['availability_365'])

new_df.number_of_reviews += 0.000000001
new_df['number_of_reviews'] = np.log10(new_df['number_of_reviews'])



In [9]:
##model 1 all variable
Model1  = new_df[['new_host_acceptance_rate','host_is_superhost','room_type_cat','bedrooms'\
                  ,'beds','number_of_reviews','availability_365','neighbourhood_cleansed_Cat','neighbourhood_group_cleansed_Cat']]
##Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat
Model2  = new_df[['room_type_cat','bedrooms'\
                  ,'number_of_reviews','neighbourhood_cleansed_Cat','neighbourhood_group_cleansed_Cat']]
##model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat
Model3  = new_df[['room_type_cat','number_of_reviews','neighbourhood_group_cleansed_Cat']]
y = new_df['price'].values
y = np.log10(y)


In [10]:
# Import train_test_split from sklearn
from sklearn.model_selection import train_test_split
m1_train, m1_test,m2_train,m2_test,m3_train,m3_test,y_train,y_test = train_test_split(Model1,Model2,Model3,y, test_size = 0.2)

In [11]:
# Check the sample sizes for salesprice
print("Train Set :", m1_train.shape, y_train.shape)
print("Test Set  :", m1_test.shape, y_test.shape)
# Check the sample sizes overallqual
print("Train Set :", m2_train.shape, y_train.shape)
print("Test Set  :", m2_test.shape, y_test.shape)
# Check the sample sizes year built
print("Train Set :", m3_train.shape, y_train.shape)
print("Test Set  :", m3_test.shape, y_test.shape)

Train Set : (2936, 9) (2936,)
Test Set  : (734, 9) (734,)
Train Set : (2936, 5) (2936,)
Test Set  : (734, 5) (734,)
Train Set : (2936, 3) (2936,)
Test Set  : (734, 3) (734,)


## Model 1  all variable

In [12]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(m1_train, y_train)
# Predicting the Test set results
y_pred = lr.predict(m1_test)

In [13]:
model1df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_pred, 0)})
model1df.head(10)

,Actual,Predicted
0,99.0,171.0
1,70.0,155.0
2,139.0,193.0
3,69.0,111.0
4,119.0,52.0
5,60.0,68.0
6,380.0,310.0
7,120.0,239.0
8,313.0,146.0
9,1600.0,164.0


In [14]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score

print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m1_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, lr.predict(m1_train), multioutput='variance_weighted'), 2))
print('R2 score test:', np.round(r2_score(y_test, lr.predict(m1_test), multioutput='variance_weighted'), 2))

Price mean: 2.13
Price std: 0.37
RMSE: 0.28
R2 score train: 0.35
R2 score test: 0.36


## Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat

In [15]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(m2_train, y_train)
# Predicting the Test set results
y_pred2 = lr.predict(m2_test)

In [16]:
model2df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_pred2, 0)})
model2df.head(10)

,Actual,Predicted
0,99.0,170.0
1,70.0,154.0
2,139.0,210.0
3,69.0,112.0
4,119.0,50.0
5,60.0,73.0
6,380.0,308.0
7,120.0,269.0
8,313.0,141.0
9,1600.0,159.0


In [17]:
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m2_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, lr.predict(m2_train), multioutput='variance_weighted'), 2))
print('R2 score test:', np.round(r2_score(y_test, lr.predict(m2_test), multioutput='variance_weighted'), 2))

Price mean: 2.13
Price std: 0.37
RMSE: 0.28
R2 score train: 0.35
R2 score test: 0.35


## Model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat

In [18]:
# Fitting Multiple Linear Regression to the Training set
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(m3_train, y_train)
# Predicting the Test set results
y_pred3 = lr.predict(m3_test)

In [19]:
model3df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_pred3, 0)})
model3df.head(10)

,Actual,Predicted
0,99.0,176.0
1,70.0,176.0
2,139.0,189.0
3,69.0,117.0
4,119.0,72.0
5,60.0,80.0
6,380.0,256.0
7,120.0,256.0
8,313.0,170.0
9,1600.0,183.0


In [20]:
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(m3_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, lr.predict(m3_train), multioutput='variance_weighted'), 2))
print('R2 score test:', np.round(r2_score(y_test, lr.predict(m3_test), multioutput='variance_weighted'), 2))

Price mean: 2.13
Price std: 0.37
RMSE: 0.29
R2 score train: 0.31
R2 score test: 0.3


## Using Random forest to optimize

## Model 1

In [21]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m1_train, y_train)
# Predicting the Test set results
y_predr1 = rfr.predict(m1_test)
df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_predr1, 0)})
df.head(10)

,Actual,Predicted
0,99.0,111.0
1,70.0,85.0
2,139.0,179.0
3,69.0,95.0
4,119.0,167.0
5,60.0,80.0
6,380.0,456.0
7,120.0,135.0
8,313.0,203.0
9,1600.0,172.0


In [22]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m1_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, rfr.predict(m1_train), multioutput='variance_weighted'), 2))
print('R2 score test:', np.round(r2_score(y_test, rfr.predict(m1_test), multioutput='variance_weighted'), 2))

Price mean: 2.13
Price std: 0.37
RMSE: 0.24
R2 score train: 0.69
R2 score test: 0.55


## Model 2 room_type_cat, bedroom,number_of_reviews,neighbourhood_cleansed_cat, neighbourhood_group_cleansed_cat

In [23]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m2_train, y_train)
# Predicting the Test set results
y_predr2 = rfr.predict(m2_test)
df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_predr2, 0)})
df.head(10)

,Actual,Predicted
0,99.0,134.0
1,70.0,82.0
2,139.0,197.0
3,69.0,71.0
4,119.0,49.0
5,60.0,77.0
6,380.0,322.0
7,120.0,169.0
8,313.0,228.0
9,1600.0,136.0


In [24]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m2_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, rfr.predict(m2_train), multioutput='variance_weighted'), 2))
print('R2 score test:', np.round(r2_score(y_test, rfr.predict(m2_test), multioutput='variance_weighted'), 2))

Price mean: 2.13
Price std: 0.37
RMSE: 0.26
R2 score train: 0.61
R2 score test: 0.46


## Model 3 room_type_cat,number_of_reviews,neighbourhood_group_cleansed_Cat

In [25]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m3_train, y_train)
# Predicting the Test set results
y_predr3 = rfr.predict(m3_test)
df = pd.DataFrame({'Actual': np.round(10 ** y_test, 0), 
                   'Predicted': np.round(10 ** y_predr3, 0)})
df.head(10)

,Actual,Predicted
0,99.0,132.0
1,70.0,100.0
2,139.0,214.0
3,69.0,120.0
4,119.0,68.0
5,60.0,85.0
6,380.0,255.0
7,120.0,255.0
8,313.0,333.0
9,1600.0,201.0


In [26]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import r2_score
print('Price mean:', np.round(np.mean(y), 2))  
print('Price std:', np.round(np.std(y), 2))
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, rfr.predict(m3_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, rfr.predict(m3_train), multioutput='variance_weighted'), 2))
print('R2 score test:', np.round(r2_score(y_test, rfr.predict(m3_test), multioutput='variance_weighted'), 2))

Price mean: 2.13
Price std: 0.37
RMSE: 0.3
R2 score train: 0.37
R2 score test: 0.28


In [27]:
import pickle
rfr = RandomForestRegressor(max_depth=8, n_estimators = 100, random_state = 0)
rfr.fit(m2_train, y_train)
#make pickle file for our model
pickle.dump(rfr,open("model.pkl", 'wb'))